# Time twice

# To objects

In [2]:
import re
import json
from deep_translator import GoogleTranslator as google_translator

translator = google_translator(source='auto', target='ru')  

class Lines:
    '''
    nextName, readLines, read, write, writeLines, mIntervals, , ,
    '''
    name = ''
    nameMask = ''
    lines = []
    intervalLines = [] 
    text = ""
    lines2 = [] # копия входных данных
    objLines = []
    enRuObjLines = []
    objText = ''
    enLines = []  
    ruLines = []
    enRuLines = []

    def __init__(self, name):
        self.name = name
        self.nameMask = self.nextName(self.name)

    def mEnRuObjLines(self, objLines, enRuLines, enRuObjLines):
        def find(a, l):
            r = ''
            l2 = list(iter(filter(lambda x: x[0] == a, l)))
            if len(l2) > 0:
                r = l2[0][1]
            return r


        for e in objLines:
            enRuObjLines.append(e)

        for e in enRuObjLines:
            li = []
            for e2 in e['en']:
                if len( e2 ) > 0:
                    li.append(find(e2, enRuLines))
                
            if len(li) == 0:
                e['ru'] = [""]
            else:
                e['ru'] = li
                

                pass
        pass

    def mTranslate(self, vStrings, vCouples):
        textEn = '\n'.join(vStrings)
        textRu = translator.translate(textEn)
        ruStrings = textRu.split('\n')
        k = 0
        for e in ruStrings:
            vCouples.append([vStrings[k], e])
            k += 1
        pass

    def mTranslateBig(self, vStrings, vCouples):
        max = 3600
        allParts = []
        sum = 0
        k = 0
        parts = []
        for e in vStrings:
            sum += len(e)
            parts.append(e)
            if sum > max or k == len(vStrings) - 1:
                allParts.append(parts)
                sum = 0
                parts = []
            k += 1

        for parts in allParts:
            self.mTranslate(parts, vCouples)


            pass



        pass




    def getEnLines(self, objLines, enLines):
        enLines1 = []
        for e in self.objLines:
            for e2 in e['en']:
                enLines1.append(e2)
        
        enLines2 = []
        begin = True
        for e in enLines1:
            if len(e) == 0 and begin:
                pass
            else:
                begin = False
                enLines2.append(e)

        enLines2.reverse()
        enLines3 = []
        begin = True
        for e in enLines2:
            if len(e) == 0 and begin:
                pass
            else:
                begin = False
                enLines3.append(e)

        enLines3.reverse()

        for e in enLines3:
            enLines.append(e)


        



    def nextName(self, name):
        name2 = ''
        patten = r'^\w\d\d'
        match = re.search(patten, name)
        name2 = match[0] if match else 'w000'
        return name2

    def readLines(self, name, items):
        with open(self.name, 'r', encoding='utf8') as file:
            for line in file:
                line2 = line.strip()
                len(line2) == 0 or items.append(line2)
       

    def read(self, name):
        self.lines = []
        self.readLines(name, self.lines)
        self.text = '\n'.join(self.lines)
        return self.text

    def write(self, name, text):
        with open(name, 'w', encoding='utf8') as file:
            file.write(text)

    def writeLines(self, name, items):
        text = '\n'.join(items)
        with open(name, 'w', encoding='utf8') as file:
            file.write(text)

    def mIntervals(self, l1, l2):
        '''
        [1,,2,,3] -> [1,2,,2,3,,3,3]
        '''
        l1.reverse()

        sPrev = ''
        patten = '^\d+\.\d+$'
        for e in l1:
            match = re.search(patten, e)
            if match:
                sPrev = e
                break

        k = 0
        for e in l1:
            match = re.search(patten, e)
            if match:
                l2.append(sPrev)
                sPrev = e
            l2.append(e)

        l1.reverse()
        l2.reverse()
        pass

    def mRu(self, text):
        ru = False
        ru = True if re.search('^[а-яА-Я]', text) else False
        return ru
        pass

    def mEn(self, text):
        en = False
        en = True if re.search('^[a-zA-Z]', text) else False
        return en
        pass

    def mTime(self, text):
        time = False
        patten = '^\d+\.\d+$'
        time = True if re.search(patten, text) else False
        return time




    def mObj(self, l1, l2):
        obj0 = {"time":[], "en": [], "ru": [] }
        l1.append("")

        k = 0
        obj = 0
        max = len(l1)
        en = []
        ru = []
        while k < max-2:
            notRun = True
            n = 0
            while k < max - 1 and self.mTime(l1[k]) and n < 2:
                if n == 0:
                    obj = obj0.copy()
                    obj["time"] = []
                    en = []
                    ru = []
                obj["time"].append(float(l1[k]))
                n += 1
                k += 1
                notRun = False

            while k < max - 1 and self.mEn(l1[k]):
                en.append(l1[k])
                obj["en"] = en
                k +=1
                notRun = False

            while k < max - 1 and self.mRu(l1[k]):
                ru.append(l1[k])
                obj["ru"] = ru
                k +=1
                notRun = False
            
            if len(obj['ru']) == 0:
                obj['ru'] = [""]
            if len(obj['en']) == 0:
                obj['en'] = [""]

            l2.append(obj)
            if notRun:
                k +=1

        pass

    def mObjText(self, objl):
        k = 0
        text = json.dumps([ob for ob in objl], indent=2, ensure_ascii=False)
        return text
       

lines = Lines('b02c.txt')
lines.read(lines.name)
lines.mIntervals(lines.lines, lines.intervalLines)





lines.mObj(lines.intervalLines, lines.objLines)
lines.objText = lines.mObjText(lines.objLines)
lines.write(lines.nameMask + 'b.txt', lines.objText)
lines.getEnLines(lines.objLines, lines.enLines)
lines.writeLines(lines.nameMask + 'c1.txt', lines.enLines)


lines.mTranslateBig(lines.enLines, lines.enRuLines)

lines.mEnRuObjLines(lines.objLines, lines.enRuLines, lines.enRuObjLines)
objText = lines.mObjText(lines.enRuObjLines)
lines.write(lines.nameMask + 'd.txt', objText)

print(len(lines.objText), objText[:200])


685 [
  {
    "time": [
      1.31,
      3.69
    ],
    "en": [
      "Welcome to Jeremy’s IT Lab."
    ],
    "ru": [
      "Добро пожаловать в ИТ-лабораторию Джереми."
    ]
  },
  {
    "time": [
   


https://habr.com/ru/post/349860/

## copy rename files

In [6]:
import os
import re
import shutil

class Wood:
    extension = ''
    path = ''
    inFiles = []
    filess = []
    log = ''

    def __init__(self, extension = 'srt'):
        self.extension = extension

    def setPath(self, path = ''):
        self.path = path

    def getInFiles(self):
        if self.path == '':
            files = os.listdir()
        else:
            files = os.listdir(self.path)
        files = list(filter(lambda e: re.search(r'\.mp3$', e), files))
        timeFiles = []
        for e in files:
            time = os.path.getmtime(w.path + '/' + e)
            timeFiles.append([e, time])
            timeFiles
            # print(os.path.getmtime(e))

        sortFiles = sorted(timeFiles, key = lambda e: e[1])
        self.inFiles = list(map(lambda e: e[0], sortFiles))
        pass

    def mNames(self, firstName, vFiles = []):
        vFiles = self.inFiles if len(vFiles) == 0 else vFiles
        max = len(vFiles)
        self.filess = list(map(lambda e: e, vFiles))

        pattern = r'^(([A-Za-z]+)(\d+))'
        match = re.search(pattern, firstName)
        mask = match[2]
        num = match[3]
        num = int(num)

        nNames = []
        for i  in range(num, num + max):
            sNum = f'{i:02d}'
            nNames.append(mask + sNum)
            pass

        self.filess = list(map(lambda a, b: [a, b], nNames, vFiles))

        # print(nNames)
        # print(mask, num, max, firstName)
        pass

    def copy(self, filess = []):
        filess = self.filess if len(filess) == 0 else filess
        for ee in filess:
            shutil.copy2(self.path + '/' + ee[1], self.path + '/' + ee[0] + '.mp3')
        pass

    def write(self, name, text=''):
        text0 = ''
        for e in self.filess:
            text0 += e[0]
            text0 += '\n'
            text0 += e[1]
            text0 += '\n'
            pass
        text = text0  if len(text) == 0 else text
        with open(name, 'a', encoding='utf8') as file:
            file.write(text)

        self.log += text

        # print(text)
        pass

w = Wood('srt')
w.setPath('mm')
w.getInFiles()
w.mNames('m33')
w.copy()
w.write(w.path + '/' + 'p06srt.txt')
print(w.log[:400])
# print(w.filess)


m33
Inkscape _ Draw Vector Graphics Watermelon.mp3
m34
Inkscape _ Draw a City or Town in Vector Art.mp3
m35
Inkscape _ Draw Cartoon Character Mega Man Artwork.mp3
m36
Inkscape _ Draw Shoes in Vector Art _ SVG.mp3
m37
Inkscape _ How to Draw Trees.mp3
m38
Inkscape _ Fix Thumbnail in Document Properties.mp3
m39
Inkscape _ Christmas Room at Night with Tree, Fireplace, and Presents.mp3
m40
Inkscape _ H


http://pythonicway.com/python-functinal-programming#filter

